In [107]:
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch import FloatTensor, LongTensor
import numpy as np
import pandas as pd
import time
import os
import pickle
import string
import torch.utils.data as data_utils
import psutil
from random import shuffle
from sklearn.utils import shuffle as skshuffle
torch.manual_seed(1)

# Load the data

In [108]:
X_train = pd.read_pickle('/data/WorkData/firmEmbeddings/Models/FirmEmbeddings/xtrain_firm6.pkl')
y_train = pd.read_pickle('/data/WorkData/firmEmbeddings/Models/FirmEmbeddings/ytrain_firm6.pkl')
X_val = pd.read_pickle('/data/WorkData/firmEmbeddings/Models/FirmEmbeddings/xval_firm6.pkl')
y_val = pd.read_pickle('/data/WorkData/firmEmbeddings/Models/FirmEmbeddings/yval_firm6.pkl')
X_test = pd.read_pickle('/data/WorkData/firmEmbeddings/Models/FirmEmbeddings/xtest_firm6.pkl')
y_test = pd.read_pickle('/data/WorkData/firmEmbeddings/Models/FirmEmbeddings/ytest_firm6.pkl')


# Training data

In [109]:
X_train

,LogDiff,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,0.233821,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772
1,-0.405790,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772
2,0.289227,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772
3,0.012848,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772
4,-0.078005,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772
5,0.215061,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772
6,0.052088,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772
7,0.012472,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772
8,0.105585,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772
9,0.030459,-2.373430,2.65059,-2.741800,0.732310,0.254526,0.988200,-5.675574,4.191410,1.313589,...,2.230572,1.873922,-0.102608,-3.344391,-2.285449,1.739269,2.935741,0.222669,-2.185889,2.082772


In [112]:
len(y_train.unique())

709

# Validation Data

In [113]:
X_val

,LogDiff,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,-0.158660,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663
1,0.239250,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663
2,0.099608,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663
3,-0.012193,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663
4,-0.049252,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663
5,-0.089938,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663
6,-0.044907,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663
7,0.051446,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663
8,-0.018516,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663
9,-0.088626,-3.775103,3.115883,0.316088,-1.804894,-0.221209,1.039743,-4.794109,3.835644,-0.006728,...,1.994839,-1.704414,1.066886,-0.321578,-5.871618,-1.655638,1.060439,0.643685,-3.743478,2.969663


In [114]:
len(y_val)

95985

# Test Data

In [115]:
X_test

,LogDiff,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,-0.294377,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582
1,-0.092968,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582
2,-0.397483,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582
3,0.344025,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582
4,0.013287,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582
5,0.041695,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582
6,-0.088666,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582
7,0.042128,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582
8,0.042794,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582
9,0.341214,-0.076809,1.465071,1.034053,2.458874,-1.756786,2.578092,-7.052349,7.453100,1.054856,...,2.348019,-2.862967,-1.744087,-2.369752,-5.438653,-4.03139,-0.147215,2.067681,-4.713803,0.677582


In [116]:
y_test

0          0.0
1          1.0
2          3.0
3          6.0
4        521.0
5          7.0
6          9.0
7        522.0
8         11.0
9         12.0
10        13.0
11        14.0
12       523.0
13        15.0
14        16.0
15        18.0
16        19.0
17        20.0
18        21.0
19       524.0
20       525.0
21        23.0
22        24.0
23       526.0
24        25.0
25        27.0
26        29.0
27        32.0
28        33.0
29        34.0
         ...  
34509    494.0
34510    495.0
34511    496.0
34512    497.0
34513    498.0
34514    499.0
34515    500.0
34516    650.0
34517    651.0
34518    502.0
34519    652.0
34520    503.0
34521    504.0
34522    505.0
34523    506.0
34524    507.0
34525    508.0
34526    509.0
34527    510.0
34528    511.0
34529    512.0
34530    513.0
34531    653.0
34532    654.0
34533    515.0
34534    516.0
34535    517.0
34536    518.0
34537    519.0
34538    520.0
Name: Company_id, Length: 34539, dtype: float64

In [117]:
y_val.min()

0.0

# Train the model

In [122]:
class Firm_Embedding_Model(nn.Module):
    def __init__(self, input_dim, hidden_layer_dim, embedding_dim, num_firms):
        super(Firm_Embedding_Model,self).__init__()
        
        # input is m x D
        self.linear1 = nn.Linear(input_dim,hidden_layer_dim) # D x H 
        self.dropout1 = nn.Dropout(p=0.5)
        self.linear2 = nn.Linear(hidden_layer_dim,hidden_layer_dim) # H x H
        self.dropout2 = nn.Dropout(p=0.5)
        self.linear3 = nn.Linear(hidden_layer_dim,embedding_dim)
        
        self.firm_embedding = nn.Linear(embedding_dim,num_firms) # H x F
        # the output is m x F
        
        self.init_weights()

    def forward(self, X):
        out = F.relu(self.linear1(X))
        out = self.dropout1(out)
        out = F.relu(self.linear2(out))
        out = self.dropout2(out)
        out = F.relu(self.linear3(out))
        out = self.firm_embedding(out)
        
        # now we have m x F matrix, for m data points, we can do log softmax
        log_prob = F.log_softmax(out,dim=1)
        return log_prob # for each case data, this is probability of which firm has this stock price change
    
    def init_weights(self):
        linear_layers = [self.linear1,self.linear2,self.linear3,self.firm_embedding]
        for layer in linear_layers:
            layer.weight.data.normal_(0.0,0.1)


BATCH_SIZE = 64
train_dataset = data_utils.TensorDataset(FloatTensor(X_train.values),LongTensor(y_train.values))
train_loader = data_utils.DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,shuffle=False)



INPUT_DIM = 101
HIDDEN_DIM = 50
EMBED_DIM = 50
NUMBER_FIRMS = 709
model = Firm_Embedding_Model(input_dim=INPUT_DIM,hidden_layer_dim=HIDDEN_DIM,
                        embedding_dim=EMBED_DIM,num_firms=NUMBER_FIRMS)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)


N_EPOCH = 7

#Need to udpated the TRAIN_SIZE with the size of the data
TRAIN_SIZE = 229676
print("Training data size",TRAIN_SIZE)
train_losses = []
val_losses = []

X_val_var = Variable(FloatTensor(X_val.values))

y_val_var = Variable(LongTensor(y_val))

model.eval()

y_pred_val = model.forward(X_val_var)


val_loss = criterion(y_pred_val,y_val_var)


startTime = time.time()
model.train()

for i_epoch in range(N_EPOCH):
    epoch_train_loss = 0
    num_batches_per_epoch = int(TRAIN_SIZE/BATCH_SIZE)
    for i_batch,(X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        
        X_var, y_var = Variable(X_batch),Variable(y_batch)
        
        y_pred = model.forward(X_var)
        #print("y_pred is", y_pred)
        loss = criterion(y_pred,y_var)
        loss.backward()
        
        optimizer.step()
        epoch_train_loss += loss.data[0]
        
    # after each epoch
    
    X_val_var = Variable(FloatTensor(X_val.values))
    y_val_var = Variable(LongTensor(y_val))
    
    model.eval()
    
    y_pred_val = model.forward(X_val_var)
    val_loss = criterion(y_pred_val,y_val_var)
    ave_train_loss = epoch_train_loss/num_batches_per_epoch
    print("epoch",i_epoch,"ave_train_loss",
          ave_train_loss,"validation loss:",val_loss.data[0],time.time()-startTime)
    val_losses.append(val_loss.data[0])
    train_losses.append(ave_train_loss)
    
    model.train()
    
trained_emb = model.firm_embedding.weight.data.numpy()    
pickle.dump(trained_emb,open("/data/WorkData/firmEmbeddings/Models/FirmEmbeddings/trained_emb_cat6.pkl","wb"))

Training data size 229676


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:92: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:103: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:104: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


epoch 0 ave_train_loss tensor(6.5126) validation loss: tensor(6.6468) 74.97387623786926
epoch 1 ave_train_loss tensor(6.5030) validation loss: tensor(6.6204) 160.5897810459137
epoch 2 ave_train_loss tensor(6.5032) validation loss: tensor(6.6165) 255.34561395645142
epoch 3 ave_train_loss tensor(6.5039) validation loss: tensor(6.6157) 334.28634691238403
epoch 4 ave_train_loss tensor(6.5027) validation loss: tensor(6.6156) 414.418329000473
epoch 5 ave_train_loss tensor(6.5028) validation loss: tensor(6.6156) 497.3204290866852
epoch 6 ave_train_loss tensor(6.5029) validation loss: tensor(6.6156) 575.5245780944824
